In [1]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="2kT053S4dqKLHoB6yTpk")
    project = rf.workspace("bitter-gourd-augemented-dataset").project("eyes-jp15h")
    version = project.version(3)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = YOLO("yolo11n")  # Adjust model if needed




Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to eyes-3 in yolov8:: 100%|███████████████████████████████████████████████████████████| 70264/70264 [01:50<00:00, 637.60it/s]


In [2]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "val", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(5):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Missing directory: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/val/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/test/labels

Number of images per class:
Class 0: 1799 images
Class 1: 189 images
Class 2: 370 images
Class 3: 52 images
Class 4: 49 images


In [3]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="Eyepac Org v11",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=64,  # Adjust batch size for better GPU utilization
    workers=16,  # Use multiple data loading workers
    optimizer="SGD"  # Use SGD optimizer
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=Eyepac Org v11, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=v

train: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/train/labels... 26696 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26696/26696 [00:35<00:00, 757.21it/s]


train: New cache created: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/train/labels.cache


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/valid/labels... 5971 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5971/5971 [00:11<00:00, 541.68it/s] 


val: New cache created: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/valid/labels.cache
Plotting labels to runs/detect/Eyepac Org v11/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/Eyepac Org v11
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5971       5971      0.596        0.6      0.608      0.608

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.48G    0.01294     0.4264     0.8351          2        640: 100%|██████████| 418/418 [02:02<00:00,  3.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 187/187 [01:12<00:00,  2.58it/s]


                   all       5971       5971      0.597        0.6      0.609      0.609

100 epochs completed in 5.716 hours.
Optimizer stripped from runs/detect/Eyepac Org v11/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/Eyepac Org v11/weights/best.pt, 5.5MB

Validating runs/detect/Eyepac Org v11/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 187/187 [00:42<00:00,  4.36it/s]


                   all       5971       5971      0.597      0.601      0.609      0.609
                     0       4379       4379      0.876      0.978       0.94       0.94
                     1        412        412      0.403      0.172      0.261      0.261
                     2        923        923      0.601       0.72      0.692      0.692
                     3        144        144       0.44      0.583      0.501      0.501
                     4        113        113      0.667      0.549      0.652      0.652
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/Eyepac Org v11


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/Eyepac Org/valid/labels.cache... 5971 images, 0 backgrounds, 0 corrupt: 100%|██████████| 597
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:43<00:00,  2.18it/s]


                   all       5971       5971      0.579      0.617       0.61       0.61
                     0       4379       4379      0.874      0.981       0.94       0.94
                     1        412        412      0.393      0.187      0.263      0.263
                     2        923        923      0.587      0.733      0.692      0.692
                     3        144        144      0.412      0.614      0.502      0.502
                     4        113        113       0.63      0.571      0.652      0.652
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/Eyepac Org v112
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd4d2f750a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)